In [1]:
# DO you filling missing values before spliting the dataset into train and test or after
# DO you convert to numeric before spliting the dataset into train and test or after
#enter the date variable correctly
#convert others to female later in gender
#decide on date_first_booking, but ight replace with -9999 or leave it like that 
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime

In [2]:
train = pd.read_csv('best_user_records.csv', parse_dates=['date_account_created','timestamp_first_active','date_first_booking'])
train.head()

,Unnamed: 0,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,...,-unknown-,booking_request,booking_response,click,data,message_post,modify,partner_callback,submit,view
0,36,y1rs6e7jyh,2014-01-02,2014-01-02 00:05:45,2014-01-02,MALE,39.0,basic,0,en,...,17.0,0.0,0.0,44.0,41.0,0.0,0.0,0.0,2.0,37.0
1,39,jxsooowkew,2014-01-02,2014-01-02 00:12:24,2014-01-06,-unknown-,36.0,basic,0,en,...,4.0,0.0,0.0,44.0,10.0,0.0,0.0,0.0,1.0,11.0
2,42,fu725nxa46,2014-01-02,2014-01-02 00:18:50,2014-02-28,MALE,36.0,basic,0,en,...,5.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,3.0
3,48,lnropcu3a0,2014-01-02,2014-01-02 00:23:58,2014-01-02,FEMALE,24.0,basic,0,en,...,13.0,1.0,0.0,16.0,11.0,1.0,0.0,0.0,3.0,12.0
4,49,zepgy5emhl,2014-01-02,2014-01-02 00:24:07,2014-01-02,MALE,26.0,facebook,0,en,...,4.0,1.0,0.0,1.0,7.0,2.0,0.0,0.0,3.0,16.0


In [3]:
train['timestamp_first_active_to_date'] = pd.to_datetime(train['timestamp_first_active'], format='%Y%m%d%H%M%S')


In [4]:
train['timestamp_first_active_to_date'] = train['timestamp_first_active_to_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [5]:
#impute age missing values with the median

train['age'].fillna(train['age'].median() , inplace = True)

In [6]:
#impute first_affiliate_tracked missing values with the mode

train['first_affiliate_tracked'].fillna(train['first_affiliate_tracked'].mode()[0] , inplace = True)

In [7]:
#Convert to numeric
labelencoder_X  = LabelEncoder()

train['gender'] = labelencoder_X.fit_transform(train['gender'])

In [8]:

train['language'] = labelencoder_X.fit_transform(train['language'])

In [9]:
train['affiliate_channel'] = labelencoder_X.fit_transform(train['affiliate_channel'])

In [10]:
train['affiliate_provider'] = labelencoder_X.fit_transform(train['affiliate_provider'])

In [11]:
train['first_affiliate_tracked'] = labelencoder_X.fit_transform(train['first_affiliate_tracked'])

In [12]:
train['signup_app'] = labelencoder_X.fit_transform(train['signup_app'])

In [13]:
train['first_device_type'] = labelencoder_X.fit_transform(train['first_device_type'])

In [14]:
train['first_browser'] = labelencoder_X.fit_transform(train['first_browser'])

In [15]:
train['country_destination'] = labelencoder_X.fit_transform(train['country_destination'])

In [16]:
train['signup_method'] = labelencoder_X.fit_transform(train['signup_method'])

In [17]:
#was a first booking made
train['first_booking_made'] = pd.notnull(train.loc[:, 'date_first_booking'])

In [18]:
#convert to integer 
train['first_booking_made'] = train['first_booking_made'].astype(int)

In [19]:
train['timestamp_first_active'] = train['timestamp_first_active'].astype(int)

### FEATURE ENG.

In [20]:
# date_account_created  timestamp_first_active  date_first_booking

In [21]:
#convert type to date type
train['timestamp_first_active_to_date'] = pd.to_datetime(train['timestamp_first_active_to_date'])

In [22]:
train['number_of_days_btw_first_active_first_booking'] = train['date_first_booking'] - train['timestamp_first_active_to_date']

In [23]:
train['month_of_first_active'] = train['timestamp_first_active_to_date'].apply(lambda x: x.strftime('%m'))

In [24]:
train['month_account_created'] = train['date_account_created'].apply(lambda x: x.strftime('%m'))

In [25]:
train['month_of_first_booking'] =  train['date_first_booking'].map(lambda x: x.strftime('%m') if pd.notnull(x) else '')

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
# train_without_dates = train[['gender', 'age', 'signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','first_booking_made','month_of_first_active','month_account_created','month_of_first_booking','number_of_days_btw_first_active_first_booking', 'country_destination']]
train_without_dates = train[['gender', 'age', 'signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','first_booking_made','number_of_days_btw_first_active_first_booking', 'country_destination']]

In [28]:
train_without_dates['delta_join_booking'] = train_without_dates['number_of_days_btw_first_active_first_booking'].apply(lambda x: x.days)

/Users/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
y = train_without_dates['country_destination']
X = train_without_dates.drop(['country_destination'], axis=1)

In [30]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
X_train = X_train.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)
X_test = X_test.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)

In [32]:
X = X.drop(['number_of_days_btw_first_active_first_booking','delta_join_booking'], axis=1)

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
lr = LogisticRegression()

In [37]:
fitted = lr.fit(X_train, y_train)

In [38]:
fitted.score(X_test, y_test)

0.71458678307993373

In [39]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(fitted, X, y, cv=5)

In [40]:
scores

array([ 0.81915158,  0.80669975,  0.80675609,  0.80487199,  0.82288557])

In [41]:
train.head()

,Unnamed: 0,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,...,modify,partner_callback,submit,view,timestamp_first_active_to_date,first_booking_made,number_of_days_btw_first_active_first_booking,month_of_first_active,month_account_created,month_of_first_booking
0,36,y1rs6e7jyh,2014-01-02,1388621145000000000,2014-01-02,2,39.0,0,0,4,...,0.0,0.0,2.0,37.0,2014-01-02,1,0 days,01,01,01
1,39,jxsooowkew,2014-01-02,1388621544000000000,2014-01-06,0,36.0,0,0,4,...,0.0,0.0,1.0,11.0,2014-01-02,1,4 days,01,01,01
2,42,fu725nxa46,2014-01-02,1388621930000000000,2014-02-28,2,36.0,0,0,4,...,0.0,0.0,1.0,3.0,2014-01-02,1,57 days,01,01,02
3,48,lnropcu3a0,2014-01-02,1388622238000000000,2014-01-02,1,24.0,0,0,4,...,0.0,0.0,3.0,12.0,2014-01-02,1,0 days,01,01,01
4,49,zepgy5emhl,2014-01-02,1388622247000000000,2014-01-02,2,26.0,1,0,4,...,0.0,0.0,3.0,16.0,2014-01-02,1,0 days,01,01,01
